In [1]:
# Buňka 1: Importy
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import gradio as gr
import traceback # Pro detailní výpis chyb

/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Buňka 2: Konfigurace
base_model_name_or_path = "/home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe"
lora_adapter_path = "mistral-7b-quantum-instruct-test/checkpoint-300" 

# --- Konfigurace kvantizace ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Konfigurace připravena.")
print(f"Cesta k základnímu modelu: {base_model_name_or_path}")
print(f"Cesta k LoRA adaptéru: {lora_adapter_path}")

Konfigurace připravena.
Cesta k základnímu modelu: /home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe
Cesta k LoRA adaptéru: mistral-7b-quantum-instruct-test/checkpoint-300


In [ ]:
# Buňka 3 (FINÁLNÍ OPRAVA)
print("Načítám základní model a tokenizer... Toto může trvat několik minut a spotřebuje hodně VRAM.")

try:
    # Načtení tokenizeru
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("...Tokenizer načten.")

    # Načtení základního modelu s kvantizací
    max_memory_map = {0: "8GiB", "cpu": "0GiB"} 

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path,
        quantization_config=bnb_config,
        device_map="auto",
        max_memory=max_memory_map, # Použijeme správně naformátovaný slovník
        torch_dtype=torch.bfloat16,
    )
    print("Základní model a tokenizer úspěšně načteny!")

except Exception as e:
    print("\n CHYBA PŘI NAČÍTÁNÍ ZÁKLADNÍHO MODELU!")
    traceback.print_exc()

Načítám základní model a tokenizer... Toto může trvat několik minut a spotřebuje hodně VRAM.
...Tokenizer načten.


Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.68s/it]

✅ Základní model a tokenizer úspěšně načteny!


In [4]:
# Buňka 4: Aplikace LoRA adaptéru
print("Aplikuji LoRA adaptér na základní model...")

try:
    model = PeftModel.from_pretrained(model, lora_adapter_path)
    model.eval() # Přepnutí do evaluačního módu je velmi důležité!
    
    print("✅ LoRA adaptér úspěšně aplikován a model je připraven k inferenci.")
    
    print(model)

except Exception as e:
    print("\n CHYBA PŘI APLIKACI LoRA ADAPTÉRU!")
    print("Zkontrolujte, zda je cesta 'lora_adapter_path' správná a obsahuje soubory jako 'adapter_config.json' a 'adapter_model.bin'.")
    traceback.print_exc()

Aplikuji LoRA adaptér na základní model...
✅ LoRA adaptér úspěšně aplikován a model je připraven k inferenci.
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B)

In [ ]:
# Buňka 5: Definice chatovací funkce
def chat_with_model_gradio(message, history):
    try:
        # Sestavení promptu s historií
        history_prompt_parts = []
        for user_msg, assistant_msg in history:
            history_prompt_parts.append(f"[INST] {user_msg} [/INST] {assistant_msg}</s>")
        history_prompt = "".join(history_prompt_parts)
        final_prompt = f"<s>{history_prompt}[INST] {message} [/INST]"

        # Tokenizace a zjištění délky vstupu
        inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
        input_token_len = inputs.input_ids.shape[1]

        # Generování odpovědi
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        # Robustní dekódování pouze nově vygenerovaných tokenů
        generated_tokens = outputs[0, input_token_len:]
        clean_response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        
        return clean_response.strip()

    except Exception as e:
        # Pokud cokoliv selže, vytiskneme chybu do výstupu buňky
        print("\n CHYBA BĚHEM GENEROVÁNÍ ODPOVĚDI V GRADIO FUNKCI!")
        traceback.print_exc()
        return "Omlouvám se, došlo k interní chybě. Podívejte se na výstup v Jupyter buňce pro detaily."

print(" Chatovací funkce 'chat_with_model_gradio' je definována.")

✅ Chatovací funkce 'chat_with_model_gradio' je definována.


In [ ]:
# Buňka 6 (Alternativa): Interaktivní chat přímo v notebooku

chat_history = []

print(" Chat je připraven! Můžete začít psát.")
print("Napište 'konec', 'exit' nebo quit pro ukončení chatu.")
print("-" * 50)

while True:
    # 1. Získání vstupu od uživatele
    user_input = input("Vy: ")
    if user_input.lower() in ["konec", "exit", "quit"]:
        print("Chat ukončen.")
        break
        
    # 2. Sestavení promptu s historií
    history_prompt_parts = []
    for user_msg, assistant_msg in chat_history:
        # Formát pro Mistral-Instruct
        history_prompt_parts.append(f"[INST] {user_msg} [/INST] {assistant_msg}</s>")
    history_prompt = "".join(history_prompt_parts)
    
    final_prompt = f"<s>{history_prompt}[INST] {user_input} [/INST]"
    
    # 3. Tokenizace a příprava vstupu pro model
    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
    input_token_len = inputs.input_ids.shape[1]
    
    print("\nAsistent přemýšlí...")
    
    # 4. Generování odpovědi
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # 5. Dekódování a vyčištění odpovědi
    generated_tokens = outputs[0, input_token_len:]
    clean_response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    print(f"\nAI Asistent: {clean_response}")
    print("-" * 50)
    
    # 6. Aktualizace historie konverzace
    chat_history.append((user_input, clean_response))

 Chat je připraven! Můžete začít psát.
Napište 'konec' nebo 'exit' pro ukončení chatu.
--------------------------------------------------

Asistent přemýšlí...

AI Asistent: "JDE" could stand for a few different things, including:

1. Java Development Environment - this is a software suite that developers use to create and run Java applications.
2. Joint Development Environment - this is a term used in project management to refer to the process of working together with someone else to achieve a common goal.
3. JDE (journalist) - this is a nickname for journalist Daniel Pearl, who was kidnapped by terrorists in Pakistan in 2002 and murdered in 2007.
4. JDE (company) - JDE is an Australian company that sells coffee machines and other products.
--------------------------------------------------

Asistent přemýšlí...

AI Asistent: I'm just a computer program, so I don't have feelings or emotions like a human does. However, I am here to help answer any questions you may have! How can I assi